In [1]:
!git clone https://github.com/UtkarshSrivastava05/stylesync.git
# !pip install -qq -U diffusers==0.11.1 transformers ftfy gradio accelerate

Cloning into 'stylesync'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 51 (delta 19), reused 34 (delta 9), pack-reused 0
Receiving objects: 100% (51/51), 77.95 KiB | 1.18 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
!pip install -qq -U diffusers==0.11.1 transformers ftfy gradio accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.9/524.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.1

In [3]:
import PIL
import requests
from io import BytesIO
import torch
import cv2
import gradio as gr
import numpy as np
from diffusers import StableDiffusionInpaintPipeline
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import torch.nn as nn
from flask import Flask, request, jsonify, render_template, session, redirect, url_for
from werkzeug.utils import secure_filename
from PIL import Image
import os

In [4]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://01q32c9igz-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, render_template, request, redirect, flash
import os
import io
from PIL import Image
import time

start_time = time.time()

# Additional libraries/functions (optional)
import uuid
from werkzeug.utils import secure_filename
allowed_extensions = ['jpg', 'jpeg', 'png', 'gif']  # List of allowed image extensions

# Set the device
device = "cuda"
# device = "cpu"

# model_path = "runwayml/stable-diffusion-inpainting"
model_path = "stabilityai/stable-diffusion-2-inpainting"

# Load the stable diffusion model
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
).to(device)

# Load image processing model for semantic segmentation
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")

# Define label mappings for clothing items
id2label = model.config.id2label
cloth_mapping = {
    'Upper-clothes': ['tshirt', 'shirt', 'top', 'top wear', 'jacket', 'crop top',
                      'sweater', 'cardigan', 'sweatshirt', 'hoodie', 'kurta'],
    'Pants': ['pant', 'trouser', 'jeans', 'leggings'],
    'Dress': ['dress', 'frock', 'one piece', 'long coat', 'jumpsuit']
}

# Function to find the original index based on text prompt and label mappings
def find_original_index(text_prompt, id2label, cloth_mapping):
    for index, label in id2label.items():
        if label in list(cloth_mapping.keys()):
            if any(word in text_prompt.lower() for word in cloth_mapping[label]):
                return index
    return None

# # def image_grid(imgs, rows, cols):
# #     assert len(imgs) == rows*cols

# #     w, h = imgs[1].size
# #     grid = PIL.Image.new('RGB', size=(cols*w, rows*h))
# #     grid_w, grid_h = grid.size

# #     for i, img in enumerate(imgs):
# #         grid.paste(img, box=(i%cols*w, i//cols*h))
# #     return grid

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[1].size
    grid = PIL.Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    grid_images = []

    for i in range(rows):
        for j in range(cols):
            index = i * cols + j
            grid_img = Image.new('RGB', (w, h))
            grid_img.paste(imgs[index], (0, 0))
            grid_images.append(grid_img)

    return grid_images

app = Flask(__name__,
            template_folder='/content/stylesync/templates',
            static_folder='/content/stylesync/static'
      )
app.config['UPLOAD_FOLDER'] = 'stylesync/static'  # Configure upload directory
app.config['STATIC_FOLDER'] = 'stylesync/static'

@app.route('/')
def index():
  return render_template('form.html')  # Replace 'form.html' with your template name


@app.route('/process_form', methods=['POST'])
def process_form():
  text = request.form['text_input']
  photo = request.files['photo_upload']

  # Validate file extension
  if photo.filename and photo.filename.rsplit('.', 1)[1].lower() not in allowed_extensions:
    flash('Allowed image types: jpg, jpeg, png, gif', 'error')
    return redirect(url_for('index'))

  # Save the photo with a secure filename
  filename = secure_filename(photo.filename)
  photo.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

  # Generate the complete image URL (add static route)
  image_url = url_for('static', filename=filename)
  print("image_url: ", image_url)

  return render_template('loading_2.html', text=text, image_url=image_url)

  # return render_template('redirect_message.html', text=text, image_url=image_url)

@app.route('/generate')
def generate():
    prompt = request.args.get('text')

    image_url = request.args.get('image_url')

    # Get the full file path from the image URL
    image_path = os.path.join(app.config['STATIC_FOLDER'], image_url.split('/')[-1])  # Extract filename from URL

    # # Open the image file for reading (replace with your processing logic)
    # with open(image_path, 'rb') as f:
    #     image_data = f.read()

    # image = Image.open(io.BytesIO(image_data))

    # print("image: ", image)

    # time.sleep(10)


    # # Find the original index based on the prompt
    found_index = find_original_index(prompt, id2label, cloth_mapping)

    if found_index is not None:
        print(f"Original Index found: {found_index}, Label: {id2label[found_index]}")
    else:
        print("No matching label found.")

    # # Assuming label number 4 is the label you're interested in
    desired_label = found_index

    # Read and preprocess the image
    IMAGE_PATH = image_path

    image = cv2.imread(IMAGE_PATH)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    org_img_size = image.shape[0:2]
    image = Image.fromarray(image)

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits.cpu()

    upsampled_logits = nn.functional.interpolate(
        logits,
        size=image.size[::-1],
        mode="bilinear",
        align_corners=False,
    )

    pred_seg = upsampled_logits.argmax(dim=1)[0]

    # Create a binary mask for the desired label
    mask = np.array(pred_seg == desired_label, dtype=np.uint8)

    # Convert binary mask to binary gray (0 or 255)
    binary_gray_mask = (mask * 255).astype(np.uint8)

    # Create a PIL Image from the binary gray mask
    pil_mask = Image.fromarray(binary_gray_mask, mode='L')

    # Resize images for processing
    image = image.resize((512, 512))
    mask_image = pil_mask.resize((512, 512))

    # Set parameters for outfit generation
    guidance_scale = 7.5
    num_samples = 3
    generator = torch.Generator(device="cuda").manual_seed(42)

    # Generate new outfits based on the prompt and input images
    images = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask_image,
        guidance_scale=guidance_scale,
        generator=generator,
        num_images_per_prompt=num_samples,
    ).images

    image_org = image.resize(tuple(reversed(org_img_size)))

    # insert initial image in the list so we can compare side by side
    images.insert(0, image_org)

    # Resize generated images to the original image size
    for i in range(len(images)):
        images[i] = images[i].resize(tuple(reversed(org_img_size)))

    # Display the image grid
    generated_images = image_grid(images, 1, num_samples + 1)
    image_url_output_final =[]

    for image in generated_images:
      ## saving the outfile for testing
      r_filename = f'output_image_final_{image}.jpg'
      # filename_output = secure_filename(generated_images[1].r_filename)
      image.save(os.path.join(app.config['UPLOAD_FOLDER'], r_filename))
      # image.save(os.path.join('static', r_filename))

      # Generate the complete image URL (add static route)
      image_url_output = url_for('static', filename=r_filename)
      # print("image_url_output: ", image_url_output)
      image_url_output_final.append(image_url_output)

    end_time = time.time()
    # Calculate execution time
    execution_time = end_time - start_time

    print("Execution time:", execution_time, "seconds")

    return render_template('final_result.html', images=image_url_output_final)


    # Display the image using the URL
    # return f'<img src="{image_url_output}" alt="Processed Image">'
    # return render_template('loading_2.html')

if __name__ == '__main__':
  app.run()

text_encoder/pytorch_model.fp16.safetensors not found


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:45:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:45:51] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:45:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:47:23] "POST /process_form HTTP/1.1" 200 -


image_url:  /static/WhatsApp_Image_2024-03-07_at_23.01.52.jpeg


...

Original Index found: 6, Label: Pants


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:51] "GET /generate?image_url=/static/random_person.jpg&text=cargo+army+design+pant HTTP/1.1" 200 -


Execution time: 566.4029643535614 seconds


INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:51] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:51] "GET /static/output_image_final_<PIL.Image.Image%20image%20mode=RGB%20size=736x920%20at%200x79B249E827A0>.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:51] "GET /static/output_image_final_<PIL.Image.Image%20image%20mode=RGB%20size=736x920%20at%200x79B249E827D0>.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:52] "GET /static/output_image_final_<PIL.Image.Image%20image%20mode=RGB%20size=736x920%20at%200x79B249E82800>.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:52] "GET /static/output_image_final_<PIL.Image.Image%20image%20mode=RGB%20size=736x920%20at%200x79B249E82650>.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:54:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/2024 03:56:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Mar/202